In [7]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from gpdm import GPDM
from gpdm_filter import GPDM_Filter
from gpdm_ukf import GPDM_UKF
from gpdm_pf import GPDM_PF

from dataset_utils.mocap_labels import RUN_TRIALS_TEST, WALK_TRIALS_TEST
import dataset_utils.select_joints as select_joints

In [8]:
TEST_TRIALS = RUN_TRIALS_TEST + WALK_TRIALS_TEST
TRUTH_LABELS = [1] * len(RUN_TRIALS_TEST) + [0] * len(WALK_TRIALS_TEST)

reduced_columns = TEST_TRIALS[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")


Number of DOFs: 35


In [9]:
d = 3 # latent space dimension
DOFs = len(reduced_columns)  # number of degrees of freedom

dyn_back_step = 1 # Number of time steps to look back in the dynamics GP

# Initial values for hyperparameters
y_lambdas_init = np.ones(DOFs)  # Signal standard deviation for observation GP
y_lengthscales_init = np.ones(d)  # Lengthscales for observation GP
y_sigma_n_init = 1e-2  # Noise standard deviation for observation GP

x_lambdas_init = np.ones(d)  # Signal standard deviation for latent dynamics GP
x_lengthscales_init = np.ones(dyn_back_step*d)  # Lengthscales for latent dynamics GP
x_sigma_n_init = 1e-2  # Noise standard deviation for latent dynamics GP
x_lin_coeff_init = np.ones(dyn_back_step*d + 1)  # Linear coefficients for latent dynamics GP

In [10]:
model_dir = Path().cwd() / 'models' / 'epochs_1k'
walk_config = model_dir / 'walk_gpdm_config.pth'
walk_state = model_dir / 'walk_gpdm_state.pth'

walk_gpdm =  GPDM(
        D=DOFs,
        d=d,
        dyn_target='full',
        dyn_back_step=dyn_back_step,
        y_lambdas_init=y_lambdas_init,
        y_lengthscales_init=y_lengthscales_init,
        y_sigma_n_init=y_sigma_n_init,
        x_lambdas_init=x_lambdas_init,
        x_lengthscales_init=x_lengthscales_init,
        x_sigma_n_init=x_sigma_n_init,
        x_lin_coeff_init=x_lin_coeff_init
    )
walk_gpdm.load(walk_config, walk_state, False)
#walk_gpdm_filter = GPDM_UKF(walk_gpdm)
#walk_gpdm_filter = GPDM_Filter(walk_gpdm)
walk_gpdm_filter = GPDM_PF(walk_gpdm, num_particles=100)




GPDM correctly loaded


In [11]:
run_config = model_dir / 'run_gpdm_config.pth'
run_state = model_dir / 'run_gpdm_state.pth'

run_gpdm =  GPDM(
        D=DOFs,
        d=d,
        dyn_target='full',
        dyn_back_step=dyn_back_step,
        y_lambdas_init=y_lambdas_init,
        y_lengthscales_init=y_lengthscales_init,
        y_sigma_n_init=y_sigma_n_init,
        x_lambdas_init=x_lambdas_init,
        x_lengthscales_init=x_lengthscales_init,
        x_sigma_n_init=x_sigma_n_init,
        x_lin_coeff_init=x_lin_coeff_init
    )

run_gpdm.load(run_config, run_state, False)
#run_gpdm_filter = GPDM_UKF(run_gpdm)
#run_gpdm_filter = GPDM_Filter(run_gpdm)
run_gpdm_filter = GPDM_PF(run_gpdm, num_particles=100)




GPDM correctly loaded


In [12]:
# Iterate over the test trials and classify them

for i, trial in enumerate(TEST_TRIALS):
    label = TRUTH_LABELS[i]

    # Get the data in an array format
    data_arr = trial.as_dataframe()[reduced_columns].to_numpy(dtype=np.float32)[::5, :]

    walk_gpdm_filter.reset()
    run_gpdm_filter.reset()

    # Run each data point through the GPDM filter
    for timestep in range(data_arr.shape[0]):
        data_point = data_arr[timestep, :]
        walk_ll = walk_gpdm_filter.update(data_point)
        if walk_ll is None:
            walk_ll = walk_gpdm_filter.log_likelihood()
        run_ll = run_gpdm_filter.update(data_point)
        if run_ll is None:
            run_ll = run_gpdm_filter.log_likelihood()


    # Classify the trial using the final log likelihoods
    if walk_ll > run_ll:
        predicted_label = 0
    else:
        predicted_label = 1

    print(f"Trial {i}: True label: {label}, Predicted label: {predicted_label}, Walk LL: {walk_ll}, Run LL: {run_ll}")


Trial 0: True label: 1, Predicted label: 1, Walk LL: -6.273519586687823, Run LL: 61.09098198272771
Trial 1: True label: 1, Predicted label: 1, Walk LL: -22.962889988410176, Run LL: 29.299724731379996
Trial 2: True label: 1, Predicted label: 1, Walk LL: -32.924133398943866, Run LL: 27.785042303701687
Trial 3: True label: 1, Predicted label: 1, Walk LL: -15.506375130597595, Run LL: 85.24944135068449
Trial 4: True label: 1, Predicted label: 1, Walk LL: -64.53775813888984, Run LL: 55.42569601609343
Trial 5: True label: 1, Predicted label: 1, Walk LL: -44.75211643456906, Run LL: 2.3045191406825376
Trial 6: True label: 1, Predicted label: 0, Walk LL: 17.6013023563533, Run LL: 10.082493496068288
Trial 7: True label: 1, Predicted label: 1, Walk LL: 37.00217457815722, Run LL: 80.44115017280508
Trial 8: True label: 1, Predicted label: 0, Walk LL: 24.51303758825931, Run LL: 2.019876338476829
Trial 9: True label: 1, Predicted label: 0, Walk LL: 16.219191199775672, Run LL: -25.452535517711723
Trial